In [137]:
import sys
sys.path.insert(0, '../') 

In [138]:
import rdflib
from wlkernel import *
import random
from itertools import chain
from pathlib import Path
from pprint import pprint
from collections import Counter

In [139]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [140]:
# there is only one instance that belong to the class id5instance, then we remove it
#Counter([t[1] for t in all_instances]).most_common()  # command to verify it, use it after all_instances definition
rdf_graph.remove( (
    rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1959instance'),
    rdflib.term.URIRef('http://swrc.ontoware.org/ontology#affiliation'),
    rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance')
))

In [141]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
all_instances = [ (t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) ) ]
assert len(all_instances) == len(set(all_instances))
print(len(all_instances))

177


In [142]:
n = 60
random.seed(42)
instances = random.sample(all_instances, n)
instances_id = [x[0] for x in instances]
instances_class = [x[1] for x in instances]
# print('Amount of the values of the classes:\n\t')
# pprint(Counter(instances_class).most_common())

In [143]:
employs_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#employs')
head_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#head')
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')
predicates_to_remove = [employs_uri, head_uri, member_uri, affiliation_uri]
for predicate in predicates_to_remove:
    rdf_graph.remove( (None, predicate, None) )

In [144]:
instances_id = [str(x) for x in instances_id]
instances_class = [str(x) for x in instances_class]

In [145]:
######## number of cores: 12 | max_depth: 2 | iterations: 1 | seed: 42
########
#### number of instances: 5
########
# SEQ: 29.8 s ± 382 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# PAR: 10.9 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# SPEEDUP: 2.7339
########
#### number of instances: 10
########
# SEQ: 1min 6s ± 501 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# PAR:   28.5s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# SPEEDUP: 2.3571
########
#### number of instances: 20
########
# SEQ: 4min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# PAR: 1min 51s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SEEPDUP: 2.4324
########
#### number of instances: 30
########
# SEQ: 14min 32s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# PAR: 5min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SPEEDUP: 2.7949
########
#### number of instances: 40
########
# SEQ: ???
# PAR: 8min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SPEEDUP: ???
########
#### number of instances: 60
########
# SEQ: ???
# PAR: 24min 20s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SPEEDUP: ???
########

In [146]:
# %timeit -n 1 -r 1 kernel_matrix = compute_kernel_matrix(rdf_graph, instances_id, 2, 1)

In [147]:
# %timeit -n 1 -r 1 kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)
kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   58.5s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

In [148]:
# kernel_matrix == kernel_matrix_par

In [232]:
# import pandas as pd
# X_df = pd.DataFrame({x: y for x, y in list(enumerate(X))})

In [265]:
X = kernel_matrix_par
y = instances_class

In [266]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)
for i in range(len(X_sc)):
    X_sc[i][i] = 0

In [267]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.50, random_state=42)

In [268]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=3)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [269]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [270]:
y_pred_train_rf = rf_clf.predict(X_train)
y_pred_test_rf = rf_clf.predict(X_test)

In [271]:
y_pred_train_tree = tree_clf.predict(X_train)
y_pred_test_tree = tree_clf.predict(X_test)

In [272]:
from sklearn.metrics import accuracy_score
print('Random Forest - 200 trees:')
print('Train accuracy: ', accuracy_score(y_train, y_pred_train_rf))
print('Test accuracy: ', accuracy_score(y_test, y_pred_test_rf))
print('Decision Tree:')
print('Train accuracy: ', accuracy_score(y_train, y_pred_train_tree))
print('Test accuracy: ', accuracy_score(y_test, y_pred_test_tree))

Random Forest - 200 trees:
Train accuracy:  0.9
Test accuracy:  0.5333333333333333
Decision Tree:
Train accuracy:  1.0
Test accuracy:  0.6666666666666666
